In [2]:
!python --version

Python 3.10.12


In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse
import pickle
parser = argparse.ArgumentParser(description='This is a demonstration program')

import os
import re

In [3]:
checkpoint_path='/content/drive/MyDrive/checkpoints/'

In [2]:
# prompt: mount google drive

#from google.colab import drive
drive.mount('/content/drive')


NameError: name 'drive' is not defined

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 64
block_size = 128 # text length
max_iters = 10000  # number of times that batch run
learning_rate = 2e-5
eval_iters = 1000
n_embd = 384  #embedding size
n_head = 4
n_layer = 4
dropout = 0.2

In [ ]:
pip install datasets

# data preprocess

In [ ]:
import datasets

ds_sft = datasets.load_dataset('TigerResearch/pretrain_en', split='train[:1%]')

In [6]:
# prompt: just download a part of hugging face dataset

!pip install datasets
ds_sft = datasets.load_dataset('TigerResearch/pretrain_en', split='train[:10%]')


NameError: name 'ds_sft' is not defined

In [ ]:
# prompt: give me a batch of this dataset

# Create a DataLoader instance to generate batches of data
data_loader = torch.utils.data.DataLoader(
    ds_sft['train'],
    batch_size=batch_size,
    shuffle=True
)

# Iterate over the DataLoader to get a batch of data
for batch in data_loader:
    # Print the batch data
    print(batch)
    break


In [ ]:
# prompt: create a masked language model pretrain function

def pretrain_mlm(model, batch, optimizer):
    # Set the model to training mode
    model.train()

    # Get the input tokens and labels
    x = batch['input_ids'].to(device)
    y = batch['labels'].to(device)

    # Generate random masks
    mask = torch.rand(x.shape) < 0.15
    mask = mask & (x != 0)  # Don't mask special tokens

    # Replace masked tokens with a special token
    x_masked = x.clone()
    x_masked[mask] = 0

    # Get the logits from the model
    logits = model(x_masked)

    # Calculate the loss
    loss = F.cross_entropy(logits.view(-1, vocab_size), y.view(-1))

    # Perform backpropagation and update the model parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()


## define a the vocaulary list from target files, and turn chars into indice


1.   this is a char level encode





In [5]:
# prompt: copy file in drive/MyDrive/pretrain to content

!cp -r /content/drive/MyDrive/pretrainfiles  /content


In [6]:
vocab_file="/content/pretrainfiles/vocab.txt"

In [7]:
chars = ""
with open(vocab_file, 'r', encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))

vocab_size = len(chars)


string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])



In [8]:
trainfile = "/content/pretrainfiles/output_opentex.txt"
valfile = "/content/pretrainfiles/output_opentex.txt"

## get batches

In [9]:
# memory map for using small snippets of text from a single file of any size
# and it doesn't read in the whole file, good to deal with large files
def get_random_chunk(split):
    filename = trainfile if split == 'train' else valfile
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            # ?? the files are stored with index, to save space, what if use index without decode and encode
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')

            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)

    return data


def get_batch(split):
    data = get_random_chunk(split)
    # for each sample, it is a random block of the chunked data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    # the target is the next token of a seq
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# model

1.   framwork
2.   decoder
3.   feedforward
4.   head



## framework

In [10]:
#framwork
class GPTLanguageModel(nn.Module):
    #class of nn.module means the parameters could be trained
    def __init__(self, vocab_size):
        super().__init__()
        #embedding(a,b) create a mapping to dimention b for each a
        #这里embedding table是一个把所有vocab映射到embedding的表，所以n_embd是只embedding的长度
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        #this is nlayers of decoder ,sequential means they are connencted on after another
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        # final layer norm
        self.ln_f = nn.LayerNorm(n_embd)
        # after norm, convert to dimention of vocab, ready for softmax to get prob for each vocab
        # 这里的vocab可以是任意粒度，char,word,subword不一定是单词
        self.lm_head = nn.Linear(n_embd, vocab_size)

        #?? directily execute when create the model instance
        self.apply(self._init_weights)

    def _init_weights(self, module):
        #initialze weights of network with normal distribution
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)

        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        #?? why not intilize bert blocks weights like kqv

    def forward(self, index, targets=None):
        B, T = index.shape
        # input index is index of text

        # idx and targets are both (B,T) tensor of integers
        # 这是把输入的text index转成embedding
        tok_emb = self.token_embedding_table(index) # (B,T,C,   = batch size , text length, 1 ；since index has length 1  )
        # 这是输入的每个position生成一个embendding， 所以入参是 arange（textlength）
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)


        #each sentence in batch has same position embedding
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        # for generate procedure, do not need target
        if targets is None:
            loss = None
        else:
        # train procedure
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            # since this model is train with seq lenth of block size
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            #print(logits.shape())
            # focus only on the last time step
            logits = logits[:, -1, :] # turn (B,T,C)  to  (B,1, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C), here ,c is the distribution of each vocab
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index


## decoder

In [11]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        # heac_size could be defined in other ways
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        #layernorm(x),x可以是个list，如果是一个int，则对最后一个维度
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        # res first then norm
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

## feedforward

In [12]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            ##??expand size of e_embd to 4 * n_embd then compress
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

## head

In [13]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()

        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        #after register, doesn't have to create this mask matrix every time
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        #?? what is the k.shape item
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out


## claim

In [17]:
# prompt: I have model checkpoint in /content/checkpoints/   dir ,   the name of checkpoint file contains the iteration numbers  ,forexample, "checkpoint_1000.pt" means it has trained 1000 iters , load the latest checkpoint to model ,and also get the iteration number

import os
import re

def load_latest_checkpoint(model, checkpoint_path):
  # Get a list of all checkpoint files in the directory
  checkpoint_files = os.listdir(checkpoint_path)

  # Find the checkpoint file with the highest iteration number
  latest_checkpoint = max(checkpoint_files, key=lambda f: int(re.findall(r'\d+', f)[0]))

  # Load the checkpoint
  checkpoint = torch.load(os.path.join(checkpoint_path, latest_checkpoint))

  # Load the model state dictionary
  model.load_state_dict(checkpoint)

  # Get the iteration number
  iteration = int(re.findall(r'\d+', latest_checkpoint)[0])

  return model, iteration


In [18]:
model = GPTLanguageModel(vocab_size)
#cur_iter=0  # if first trained
# print('loading model parameters...')
# with open('model-01.pkl', 'rb') as f:
#     model = pickle.load(f)
# print('loaded successfully!')


model,cur_iter= load_latest_checkpoint(model,checkpoint_path)

model = model.to(device)


## eval function

In [15]:
#means this process doesnt update gradient
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    #compute loss during tarin every eval_iters for both train and del datasets
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

## optimizer


In [17]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


# train

In [ ]:

for iter in range(max_iters):
    if iter % 50 ==0:
      print(iter)

    if iter % eval_iters  == 0 and iter > 0 :
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


In [25]:
# prompt: I have a model from nn.module, give me a train function, every 2000 step store the current step model to checkpoint, replace the previous one . and the checkpoint file should named by the step counts

def train(model, optimizer, max_iters):
    for iter in range(max_iters):
        iter = iter+cur_iter+1
        if iter % 2000 == 0 and iter>0 :
            # Save the model checkpoint
            ckpt_path =  checkpoint_path + f"checkpoint_{iter}.pt"
            torch.save(model.state_dict(), ckpt_path)
            print(f"Saved checkpoint at {ckpt_path}")

        if iter % 50 ==0:
              print(iter)

        if iter % eval_iters  == 0 and iter > 0 :
          losses = estimate_loss()
          print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

        # Perform training steps
         # sample a batch of data
        xb, yb = get_batch('train')

        # evaluate the loss
        logits, loss = model.forward(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
    print(loss.item())


In [ ]:
train(model,optimizer,max_iters)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
step: 1000, train loss: 2.238, val loss: 2.216
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
Saved checkpoint at /content/drive/MyDrive/checkpoints/checkpoint_2000.pt
2000
step: 2000, train loss: 2.140, val loss: 2.140
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
step: 3000, train loss: 2.040, val loss: 2.045
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
Saved checkpoint at /content/drive/MyDrive/checkpoints/checkpoint_4000.pt
4000
step: 4000, train loss: 1.980, val loss: 1.975
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
step: 5000, train loss: 1.905, val loss: 1.932
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
Saved checkpoint at /content/drive/MyDrive

In [ ]:
#with open('model-01.pkl', 'wb') as f:
#    pickle.dump(model, f)
#print('model saved')

In [22]:
# prompt: create a checkpoints dir in /content/checkpoints

#!mkdir -p /content/drive/MyDrive/checkpoints


# infer

In [20]:
prompt = 'How are you doing today?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

How are you doing today?

profficeght think is per‘wer gooast on vered-devattebling the thirou\ firsondite atvew of steplice
